In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from embeddings import EmbeddingSummation

In [2]:
train = pd.read_excel('../Data_Train.xlsx')
train.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [3]:
albert = EmbeddingSummation('bert-base-cased').eval()#.cuda()
albert.save_embeddings()
params = [p for p in albert.parameters()]

Saved embeddings at embeddings_bert-base-cased.pt


In [4]:
albert = EmbeddingSummation('bert-base-cased', fromDownload=False).eval()
params2 = [p for p in albert.parameters()]
with torch.no_grad():
    for p, q in zip(params, params2):
        x = p-q
        print(x.sum())

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


In [5]:
dataset = list(zip(train.Synopsis, train.Title))
loader = DataLoader(dataset, batch_size=64)
outs = []
with torch.no_grad():
    for title, synopsis in tqdm(loader):
        out = torch.cat([
            albert(title),
            albert(synopsis),
        ], dim=-1)
        outs.append(out.cpu())
        torch.cuda.empty_cache()
outs = torch.cat(outs)
print("Output obtained successfully, shape =", outs.shape)


Output obtained successfully, shape = torch.Size([6237, 1536])
